In [78]:
import sqlite3
from urllib.request import urlopen, urlretrieve
from sqlite3 import Error
import os
import pickle

In [61]:
db_file = r"C:\Users\Adam\Documents\Xscholar_works.db"
conn = sqlite3.connect(db_file)

In [30]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None


In [31]:
a_connection = create_connection(db_file)

In [74]:
create_table_stmt = """ CREATE TABLE IF NOT EXISTS publications (
                                        id integer PRIMARY KEY,
                                        title text NOT NULL,
                                        abstract text,
                                        author text
                                        collab text,
                                        subject text,
                                        year text,
                                        full_text text DEFAULT NULL
                                    ); """

In [75]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [67]:
def drop_table(conn,delete_table_sql):
     
    try:    
        c = conn.cursor()
        c.execute(delete_table_sql)
    except Error as e:
        print(e)


In [ ]:
create_table(conn, create_table_stmt)

In [73]:
drop_table(conn, "DROP TABLE publications")

In [7]:
from selenium import webdriver
import os
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [8]:
options = Options()
options.add_experimental_option("prefs", {
  "download.default_directory": os.getcwd() + "\Downloads",
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})

In [4]:
def scholar_works_scraper():
#Instatiate the webdriver and set to implicitly wait for page objects to load
    driver = webdriver.Chrome(options=options)
#     driver.implicitly_wait(5) # seconds loading implicitly

    #Set driver to go to start_page and search a given name
    source_page = "http://scholarworks.csun.edu/handle/10211.2/309"    
    driver.get(source_page)
    
    authors_link_objects = driver.find_elements_by_xpath('//*[@id="aspect_discovery_Navigation_list_discovery"]//a')

    authors = []
    pub_count = []
    research_links = []
    view_more_links = []
    
    for obj in authors_link_objects:
        if obj.text == '... View More':
            break
        else:
            split = obj.text.split('(')
            authors.append(split[0].strip())
            pub_count.append(split[1].replace(')',''))
            research_links.append(obj.get_attribute('href'))
            
    for link in research_links:
        driver.get(link)
        if driver.find_elements_by_xpath('//*[@id="aspect_discovery_Navigation_list_discovery"]/ul/li[2]/ul/li[6]/a'):
            view_more = driver.find_element_by_xpath('//*[@id="aspect_discovery_Navigation_list_discovery"]/ul/li[2]/ul/li[6]/a')
            view_more_links.append(view_more.get_attribute('href'))
        else:
            view_more_links.append('No subjects listed')
            
    return authors, pub_count, research_links, view_more_links

In [5]:
authors, pub_count, research_links, view_more_links = scholar_works_scraper()

In [24]:
def collect_subj_links(view_more_links):
    
    driver = webdriver.Chrome(options=options)
    driver.get(view_more_links)
    subject_obj = driver.find_elements_by_xpath('//*[@id="aspect_discovery_SearchFacetFilter_table_browse-by-subject-results"]/tbody/tr/td/a')
    subject_titles = []
    subject_links = []

    for obj in subject_obj:
        subject_titles.append(obj.text)
        subject_links.append(obj.get_attribute("href"))

    if driver.find_elements_by_xpath('//a[@class="next-page-link"]'):
        more_results = True
        next_button =  driver.find_element_by_xpath('//a[@class="next-page-link"]')
        webdriver.ActionChains(driver).move_to_element(next_button).click(next_button).perform()
    else:
        more_results = False

    while more_results:
        subject_obj = driver.find_elements_by_xpath('//*[@id="aspect_discovery_SearchFacetFilter_table_browse-by-subject-results"]/tbody/tr/td/a')

        for obj in subject_obj:
            subject_titles.append(obj.text)
            subject_links.append(obj.get_attribute("href"))


        if driver.find_elements_by_xpath('//a[@class="next-page-link"]'):
            #Click the next button and reselect the next button
            if more_results:
                next_button =  driver.find_element_by_xpath('//a[@class="next-page-link"]')
                webdriver.ActionChains(driver).move_to_element(next_button).click(next_button).perform()
            if driver.find_elements_by_xpath('//a[@class="next-page-link"]'):
                next_button =  driver.find_element_by_xpath('//a[@class="next-page-link"]')
            else:
                more_results = False
                
    subj = []
    subj_count = []
    
    for title in subject_titles:
        split = title.split('(')
        subj.append(split[0].strip())
        subj_count.append(split[1].replace(')',''))
        
    subject_titles = subj
    return subject_titles, subj_count, subject_links

In [ ]:
for link in view_more_links:
    subject_titles, subj_count, subject_links = collect_subj_links(link)

In [25]:
subject_titles, subj_count, subject_links = collect_subj_links(view_more_links[0])

In [26]:
subject_titles

['2-dimiensional systems',
 'A. Perovskite NiCrO',
 'ab initio',
 'Ab initio calculations',
 'ab initio calculations',
 'Ab Initio Electronic Structure Calculations',
 'accelerated molecular dynamics',
 'Actinide',
 'Al',
 'Al/graphite battery',
 'alloys',
 'aluminum',
 'Aluminum and silver',
 'Anionic Conjugated Polyelectrolytes',
 'antiferromagnetic semiconductor',
 'atom molecular dynamics',
 'Atomic',
 'Atomic displacement',
 'atomic relaxation',
 'Atomic structure',
 'Atomistic model',
 'Atomistic Simulation',
 'Atomistic simulation',
 'Atomistic simulations',
 'Band gap',
 'band structure',
 'Bande conduction',
 'Brillouin zone sampling',
 'bulk diffusion barrier',
 'C. Structural property',
 'C60 Molecules',
 'carbon dioxide',
 'carbon monoxide',
 'Carbon nanotubes',
 'carbon nanotubes',
 'catalysts',
 'Cationic Conjugated Polyelectrolytes',
 'Cauchy-Born hypothesis',
 'Charge separation',
 'Charge transfer',
 'charge-transfer',
 'charge-transfer plasmon',
 'Chemical tunining',


In [29]:
subject_links[2]

'http://scholarworks.csun.edu/handle/10211.2/309/discover?field=subject&filtertype_0=author&filter_0=Lu%2C+Gang&filter_relational_operator_0=equals&filtertype=subject&filter_relational_operator=equals&filter=ab+initio'

In [33]:
def articles_collector(subject_links):
    driver = webdriver.Chrome(options=options)
    driver.get(subject_links)
    art_objs = driver.find_elements_by_xpath('//*[@id="aspect_discovery_SimpleSearch_div_search-results"]/ul/ul/li/div[2]/div/a')
    art_links = []
    art_titles = []
    for obj in art_objs:
        art_links.append(obj.get_attribute('href'))
        art_titles.append(obj.text)
        
    return art_links, art_titles
        

In [36]:
art_links, art_titles = articles_collector(subject_links[2])

In [37]:
art_titles

['Monte Carlo Simulations of Grain Boundary Sliding and Migration: Effect of Temperature and Vacancy',
 'Multiplane-induced widening of stacking faults in fcc metals',
 'Non-singular descriptions of dislocation cores: a hybrid ab initio continuum approach']

In [39]:
art_links[1]

'http://scholarworks.csun.edu/handle/10211.3/196549'

In [59]:
def citation_collector(art_links):
    
    driver = webdriver.Chrome(options=options)
    driver.get(art_links)
    
    date_obj = driver.find_element_by_xpath('//*[@id="aspect_artifactbrowser_ItemViewer_div_item-view"]/div[1]/div[3]/span[2]')
    date = date_obj.text
    
    abstract_obj = driver.find_element_by_xpath('//*[@id="aspect_artifactbrowser_ItemViewer_div_item-view"]/div[1]/div[4]/div')
    abstract = abstract_obj.text
    
    if driver.find_elements_by_xpath('//*[@id="aspect_artifactbrowser_ItemViewer_div_item-view"]/div[2]/div/div[3]/ul/li/a'):
        pdf_link_obj = driver.find_element_by_xpath('//*[@id="aspect_artifactbrowser_ItemViewer_div_item-view"]/div[2]/div/div[3]/ul/li/a')
        pdf_link = pdf_link_obj.get_attribute('href')
        urlretrieve(pdf_link, 'a.pdf')
        
        
        
        
        # delete the file
        os.remove('a.pdf')
        
        
    return date, abstract, html
    

In [60]:
date, abstract, html = citation_collector(art_links[1])